In [2]:
import torch
print(torch.__version__, torch.cuda.is_available())
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("torch cuda:", torch.version.cuda)

2.8.0+cu126 True
torch: 2.8.0+cu126
cuda available: True
torch cuda: 12.6


In [ ]:
!pip -q install uv

In [4]:
!pip -q uninstall -y torch torchvision torchaudio
!pip -q install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --index-url https://download.pytorch.org/whl/cu126

!pip -q install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv \
  -f https://data.pyg.org/whl/torch-2.8.0+cu126.html
!pip -q install torch-geometric

In [62]:
%rm -rf /content/qm9-egnn-tda
%cd /content
!git clone https://github.com/serafim-tkachenko/qm9-egnn-tda.git
%cd /content/qm9-egnn-tda

/content
Cloning into 'qm9-egnn-tda'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 72 (delta 25), reused 64 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (72/72), 163.29 KiB | 1.31 MiB/s, done.
Resolving deltas: 100% (25/25), done.
/content/qm9-egnn-tda


In [9]:
!uv venv
!uv pip install egnn-pytorch giotto-tda numpy tqdm scikit-learn pyyaml

Using CPython 3.12.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate
Using Python 3.12.12 environment at: /usr
Audited 6 packages in 98ms


In [1]:
import egnn_pytorch, gtda


In [63]:
!python -m src.train

Extracting data/qm9/raw/qm9_v3.zip
Processing...
Using a pre-processed version of the dataset. Please install 'rdkit' to alternatively process the raw data.
Done!
Epoch 1/10 [train]: 100% 1636/1636 [01:16<00:00, 21.30it/s, train_loss=0.151, train_mae=0.304]
Epoch 1: train_loss=0.759481 train_MAE=0.6387 val_MAE=0.4034
Saved best checkpoint: artifacts/checkpoints/best_egnn.pt
Epoch 2/10 [train]: 100% 1636/1636 [01:05<00:00, 25.08it/s, train_loss=0.17, train_mae=0.341]
Epoch 2: train_loss=0.235632 train_MAE=0.3647 val_MAE=0.3410
Saved best checkpoint: artifacts/checkpoints/best_egnn.pt
Epoch 3/10 [train]: 100% 1636/1636 [01:05<00:00, 24.96it/s, train_loss=0.107, train_mae=0.262]
Epoch 3: train_loss=0.185811 train_MAE=0.3252 val_MAE=0.3148
Saved best checkpoint: artifacts/checkpoints/best_egnn.pt
Epoch 4/10 [train]: 100% 1636/1636 [01:05<00:00, 25.03it/s, train_loss=0.147, train_mae=0.294]
Epoch 4: train_loss=0.157560 train_MAE=0.2987 val_MAE=0.2731
Saved best checkpoint: artifacts/checkpo

In [64]:
!python -m src.eval

Baseline EGNN results:
  val MAE : 0.205597
  test MAE: 0.205111
Robustness (test):
  sigma=0.000 -> test MAE=0.205111
  sigma=0.010 -> test MAE=0.205833
  sigma=0.050 -> test MAE=0.234249
  sigma=0.100 -> test MAE=0.388558
Saved:
  results/baseline_metrics.json
  results/baseline_robustness.csv
  figures/baseline_robustness.png


In [84]:
#  We should do it as TDA cache construction is time/calculation expensive operation
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
!mkdir -p /content/drive/MyDrive/qm9-egnn-tda/checkpoints
!cp -v artifacts/checkpoints/best_egnn.pt /content/drive/MyDrive/qm9-egnn-tda/checkpoints/

!mkdir -p /content/drive/MyDrive/qm9-egnn-tda/results /content/drive/MyDrive/qm9-egnn-tda/figures
!cp -v results/baseline_metrics.json results/baseline_robustness.csv /content/drive/MyDrive/qm9-egnn-tda/results/
!cp -v figures/baseline_robustness.png /content/drive/MyDrive/qm9-egnn-tda/figures/

!mkdir -p /content/drive/MyDrive/qm9-egnn-tda/data
!cp -r data/qm9 /content/drive/MyDrive/qm9-egnn-tda/data/

'artifacts/checkpoints/best_egnn.pt' -> '/content/drive/MyDrive/qm9-egnn-tda/checkpoints/best_egnn.pt'
'results/baseline_metrics.json' -> '/content/drive/MyDrive/qm9-egnn-tda/results/baseline_metrics.json'
'results/baseline_robustness.csv' -> '/content/drive/MyDrive/qm9-egnn-tda/results/baseline_robustness.csv'
'figures/baseline_robustness.png' -> '/content/drive/MyDrive/qm9-egnn-tda/figures/baseline_robustness.png'


In [89]:
!git pull

Already up to date.


In [90]:
%env QM9_ROOT=/content/drive/MyDrive/qm9-egnn-tda/data/qm9
%env TDA_CACHE_DIR=/content/drive/MyDrive/qm9-egnn-tda/tda_cache

!python -m scripts.build_tda_cache

env: QM9_ROOT=/content/drive/MyDrive/qm9-egnn-tda/data/qm9
env: TDA_CACHE_DIR=/content/drive/MyDrive/qm9-egnn-tda/tda_cache
QM9_ROOT: /content/drive/MyDrive/qm9-egnn-tda/data/qm9
TDA_CACHE_DIR: /content/drive/MyDrive/qm9-egnn-tda/tda_cache
TDA feature dim: 130
Running TDA smoke test...
smoke 0 (130,) mean= 1.9461538791656494 std= 2.0011978149414062 nonzero= 65
smoke 1 (130,) mean= 1.458345890045166 std= 1.5039222240447998 nonzero= 65
smoke 2 (130,) mean= 0.9692307710647583 std= 1.0071930885314941 nonzero= 65
Starting full TDA cache build...
Building TDA cache: 100% 130831/130831 [1:02:50<00:00, 34.70it/s]
Done. Cached to: /content/drive/MyDrive/qm9-egnn-tda/tda_cache


In [88]:
# !rm -rf /content/drive/MyDrive/qm9-egnn-tda/tda_cache
!mkdir -p /content/drive/MyDrive/qm9-egnn-tda/tda_cache

In [111]:
!git pull

Already up to date.


In [98]:
%env QM9_ROOT=/content/drive/MyDrive/qm9-egnn-tda/data/qm9
%env TDA_CACHE_DIR=/content/drive/MyDrive/qm9-egnn-tda/tda_cache
%env CKPT_DIR=/content/drive/MyDrive/qm9-egnn-tda/checkpoints
%env RESULTS_DIR=/content/drive/MyDrive/qm9-egnn-tda/results
%env FIGURES_DIR=/content/drive/MyDrive/qm9-egnn-tda/figures

!python -m src.train_fusion

env: QM9_ROOT=/content/drive/MyDrive/qm9-egnn-tda/data/qm9
env: TDA_CACHE_DIR=/content/drive/MyDrive/qm9-egnn-tda/tda_cache
env: CKPT_DIR=/content/drive/MyDrive/qm9-egnn-tda/checkpoints
env: RESULTS_DIR=/content/drive/MyDrive/qm9-egnn-tda/results
env: FIGURES_DIR=/content/drive/MyDrive/qm9-egnn-tda/figures
Epoch 1/10 [train_fusion]:   0% 0/1636 [00:00<?, ?it/s]TDA batch: torch.Size([64, 130]) mean/std: 7.11640739440918 7.282740592956543
Epoch 1/10 [train_fusion]: 100% 1636/1636 [09:05<00:00,  3.00it/s, train_loss=0.364, train_mae=0.425]
Epoch 1: train_loss=0.647572 train_MAE=0.5992 val_MAE=0.4809
Saved best fusion checkpoint: /content/drive/MyDrive/qm9-egnn-tda/checkpoints/best_fusion.pt
Epoch 2/10 [train_fusion]: 100% 1636/1636 [05:14<00:00,  5.21it/s, train_loss=0.12, train_mae=0.272]
Epoch 2: train_loss=0.214272 train_MAE=0.3485 val_MAE=0.3113
Saved best fusion checkpoint: /content/drive/MyDrive/qm9-egnn-tda/checkpoints/best_fusion.pt
Epoch 3/10 [train_fusion]: 100% 1636/1636 [04:54

In [105]:
# Just in case we run this step separately let's redefine env variables
%env QM9_ROOT=/content/drive/MyDrive/qm9-egnn-tda/data/qm9
%env TDA_CACHE_DIR=/content/drive/MyDrive/qm9-egnn-tda/tda_cache
%env CKPT_DIR=/content/drive/MyDrive/qm9-egnn-tda/checkpoints
%env RESULTS_DIR=/content/drive/MyDrive/qm9-egnn-tda/results
%env FIGURES_DIR=/content/drive/MyDrive/qm9-egnn-tda/figures

!python -m src.eval

env: QM9_ROOT=/content/drive/MyDrive/qm9-egnn-tda/data/qm9
env: TDA_CACHE_DIR=/content/drive/MyDrive/qm9-egnn-tda/tda_cache
env: CKPT_DIR=/content/drive/MyDrive/qm9-egnn-tda/checkpoints
env: RESULTS_DIR=/content/drive/MyDrive/qm9-egnn-tda/results
env: FIGURES_DIR=/content/drive/MyDrive/qm9-egnn-tda/figures
Comparison results:
  EGNN             val MAE=0.205597 test MAE=0.205111
  EGNN+TDA (FiLM)  val MAE=0.200894 test MAE=0.202298
Robustness (test):
  sigma=0.000 -> EGNN=0.205111  EGNN+TDA=0.202298
  sigma=0.010 -> EGNN=0.205833  EGNN+TDA=0.202404
  sigma=0.050 -> EGNN=0.234442  EGNN+TDA=0.213665
  sigma=0.100 -> EGNN=0.390408  EGNN+TDA=0.301380
Saved:
  /content/drive/MyDrive/qm9-egnn-tda/results/compare_metrics.json
  /content/drive/MyDrive/qm9-egnn-tda/results/compare_table.csv
  /content/drive/MyDrive/qm9-egnn-tda/results/compare_robustness.csv
  /content/drive/MyDrive/qm9-egnn-tda/figures/compare_robustness.png


In [110]:
%env QM9_ROOT=/content/drive/MyDrive/qm9-egnn-tda/data/qm9
%env TDA_CACHE_DIR=/content/drive/MyDrive/qm9-egnn-tda/tda_cache
%env CKPT_DIR=/content/drive/MyDrive/qm9-egnn-tda/checkpoints
%env RESULTS_DIR=/content/drive/MyDrive/qm9-egnn-tda/results
%env FIGURES_DIR=/content/drive/MyDrive/qm9-egnn-tda/figures

!python -m scripts.tda_visualize

env: QM9_ROOT=/content/drive/MyDrive/qm9-egnn-tda/data/qm9
env: TDA_CACHE_DIR=/content/drive/MyDrive/qm9-egnn-tda/tda_cache
env: CKPT_DIR=/content/drive/MyDrive/qm9-egnn-tda/checkpoints
env: RESULTS_DIR=/content/drive/MyDrive/qm9-egnn-tda/results
env: FIGURES_DIR=/content/drive/MyDrive/qm9-egnn-tda/figures
Saved: /content/drive/MyDrive/qm9-egnn-tda/figures/tda_pointcloud_xy.png
Saved: /content/drive/MyDrive/qm9-egnn-tda/figures/tda_betti_curves.png
